<a href="https://colab.research.google.com/github/omanofx/entregable_1/blob/Proyecto_final_Omar_Fernandez/cross_validation_Omar_Fernandez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aplicamos validación cruzada al dataset de hoteles ya procesado y depurado. Contiene todas las nuevas características creadas en el desafio de la clase 45.

In [1]:
import numpy as np
import pandas as pd

# memory management
import gc

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

import xgboost as xgb

from sklearn.metrics import mean_squared_error

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, HalvingRandomSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error, r2_score

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

## funciones propias

In [2]:
def tiene_espacios_en_blanco(columna: str):
    return any(x.isspace() for x in columna)

In [3]:
# Definir la función analizar_dataframe
def analizar_dataframe(df1: pd.DataFrame, porcentaje_tolerancia: float):
    '''
    ## Esta función se utiliza para comprobar la consistencia de los datos de un DataFrame
    ### evita invocar a .info(), .isnull()
    #### Porcentaje de tolerancia: valor porcentual que se tolera para los valores nulos de
    #### cada caracteristica del dataset. Si el valor se encuentra por sobre la tolerancia,
    #### se indica como 'BORRAR' la caracteristica.
    '''
    # Validar que porcentaje_tolerancia esté entre 0 y 100
    if not (0 <= porcentaje_tolerancia <= 100):
        print("Error: El porcentaje de tolerancia debe estar entre 0 y 100.")
        return

    porcentaje_perdidos = df1.isnull().sum() * 100 / len(df1)
    total_nulos = df1.isnull().sum()
    tipo_dato = df1.dtypes
    valores_no_nulos_por_columna = df1.count()
    descripcion = df1.describe().transpose()

    resultado_analisis = pd.DataFrame({
        'Tipo de dato': tipo_dato,
        'Total No nulos': valores_no_nulos_por_columna,
        'Total nulos': total_nulos,
        'Porcentaje Nulos': round(porcentaje_perdidos, 2),
        'Borrar': np.where(porcentaje_perdidos > porcentaje_tolerancia, 'BORRAR', ''),
        'Columna con Espacios': df1.columns.to_series().apply(lambda x: tiene_espacios_en_blanco(x))
    })

    # Agregar las columnas de la descripción al resultado_analisis
    resultado_analisis = pd.concat([resultado_analisis, round(descripcion, 2)], axis=1)

    estilo_resultado = (
        resultado_analisis.style
        .applymap(lambda x: 'background-color: red',
                  subset=pd.IndexSlice[resultado_analisis['Porcentaje Nulos'] > porcentaje_tolerancia, 'Porcentaje Nulos'])
        .background_gradient(cmap='Reds', subset=['Porcentaje Nulos'])
        .applymap(lambda x: 'background-color: red' if x else '',
                  subset=pd.IndexSlice[resultado_analisis['Columna con Espacios'], 'Columna con Espacios'])
        .background_gradient(cmap='Reds', subset=['Columna con Espacios'])
    )


    return estilo_resultado

In [4]:
def calcular_precision_modelo(y_test, y_pred):
  '''
  ## Calcula la precisión del modelo
  '''
  accuracy_puntaje = accuracy_score(y_test, y_pred)

  print("Precisión del modelo: {:.2f}%".format(accuracy_puntaje * 100))

  if accuracy_puntaje > 0.9:
      print("El modelo tiene un alto rendimiento en la clasificación.")
  else:
      print("El modelo podría necesitar mejoras para lograr un rendimiento más alto.")

## Carga del Data set

In [5]:
#dataset procesado: última version 12-2-2024
df_reservas = pd.read_csv("https://raw.githubusercontent.com/omanofx/entregable_1/Proyecto_final_Omar_Fernandez/df_reservas_ya_procesado.csv", sep = ",")

In [6]:
df_reservas_backup = df_reservas

In [7]:
# Deja unicamente las características númericas ya que las categoricas fueron convertirdas con Label Enconder.
df_reservas_numerico = df_reservas.select_dtypes(include='number')
#.sample(5000) # limito el DF

In [8]:
analizar_dataframe(df_reservas_numerico, porcentaje_tolerancia=0)

,Tipo de dato,Total No nulos,Total nulos,Porcentaje Nulos,Borrar,Columna con Espacios,count,mean,std,min,25%,50%,75%,max
is_canceled,int64,85140,0,0.000000,,False,85140.000000,0.280000,0.450000,0.000000,0.000000,0.000000,1.000000,1.000000
lead_time,int64,85140,0,0.000000,,False,85140.000000,81.020000,85.980000,0.000000,12.000000,51.000000,127.000000,709.000000
arrival_date_year,int64,85140,0,0.000000,,False,85140.000000,2016.220000,0.680000,2015.000000,2016.000000,2016.000000,2017.000000,2017.000000
arrival_date_week_number,int64,85140,0,0.000000,,False,85140.000000,26.800000,13.620000,1.000000,16.000000,27.000000,37.000000,53.000000
arrival_date_day_of_month,int64,85140,0,0.000000,,False,85140.000000,15.830000,8.840000,1.000000,8.000000,16.000000,24.000000,31.000000
stays_in_weekend_nights,int64,85140,0,0.000000,,False,85140.000000,1.020000,1.020000,0.000000,0.000000,1.000000,2.000000,16.000000
stays_in_week_nights,int64,85140,0,0.000000,,False,85140.000000,2.660000,2.020000,0.000000,1.000000,2.000000,4.000000,40.000000
adults,int64,85140,0,0.000000,,False,85140.000000,1.880000,0.500000,0.000000,2.000000,2.000000,2.000000,4.000000
children,int64,85140,0,0.000000,,False,85140.000000,0.140000,0.460000,0.000000,0.000000,0.000000,0.000000,3.000000
babies,int64,85140,0,0.000000,,False,85140.000000,0.010000,0.110000,0.000000,0.000000,0.000000,0.000000,10.000000


In [9]:
df_pca = df_reservas_numerico[['is_canceled',
'deposit_type_le',
 'lead_time',
 'adr',
 'market_segment_le',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'distribution_channel_le',
 'booking_changes',
 'previous_cancellations',
 'assigned_room_type_le',
 'reserved_room_type_le',
 'es_grupo_familiar',
 'arrival_date_year',
 'stays_in_week_nights',
 'previous_bookings_not_canceled',
 'days_in_waiting_list',
 'total_pasajeros',
 'agent',
 'mes_arribo_numero']].copy()

# Separar las características (X) y la variable objetivo si es aplicable
X = df_pca.drop('is_canceled', axis=1)
y = df_pca['is_canceled']

# Escalar las características para asegurar que tengan la misma escala
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aplicar PCA con dos componentes principales para la visualización
pca = PCA(n_components = 2)
principal_components = pca.fit_transform(X_scaled)

# Crear un DataFrame
df_principal_components = pd.DataFrame(data = principal_components, columns=['PCA_1', 'PCA_2'])
df_principal_components['is_canceled'] = y.values

df_principal_components

,PCA_1,PCA_2,is_canceled
0,-0.373175,-1.142229,1
1,0.628358,-1.238869,0
2,2.072278,0.890160,1
3,-2.048619,0.594040,1
4,-1.157241,4.062686,0
...,...,...,...
85135,-1.961006,-0.127367,1
85136,-0.399588,-0.299404,0
85137,-2.252073,-0.161166,0
85138,3.004379,2.242197,0


In [ ]:
# Junta el DF numerico con el PCA
df_regresion = pd.concat([df_reservas_numerico, df_principal_components[['PCA_1','PCA_2']]], axis=1)
df_regresion.info()

In [13]:
# probar con los features calculados con PCA
df_regresion = df_regresion.sample(5000).copy()

X = df_regresion.drop('is_canceled', axis=1)
y = df_regresion['is_canceled']

# Separación train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#model = LogisticRegression(max_iter=100, n_jobs=-1)
model= xgb.XGBClassifier(learning_rate=0.001)

# Ajustar modelo
model.fit(X_train, y_train)

# Predicciones
predicciones = model.predict(X_test)

# Llama a la función para calcular la precisión del modelo
calcular_precision_modelo(y_test, predicciones )


Precisión del modelo: 72.50%
El modelo podría necesitar mejoras para lograr un rendimiento más alto.


In [14]:
# Dividir los datos en características (X) y etiquetas (y)
X = df_regresion.drop(columns=['is_canceled'])
y = df_regresion['is_canceled']
print(X.shape, y.shape)

(5000, 32) (5000,)


In [15]:
# Separar en train y test
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape, X_test.shape)

(3500, 32) (1500, 32)


In [16]:
# Lista de hiperparametros
params_1 = {'criterion': 'gini', 'splitter': 'best', 'max_depth': 5}
params_2 = {'criterion': 'entropy', 'splitter': 'random', 'max_depth': 8}
params_3 = {'criterion': 'gini', 'splitter': 'random', 'max_depth': 10}

In [17]:
# Modelo 1
model.set_params(**params_1).fit(X_train, y_train)
print(f'Accuracy para Modelo 1 = {round(accuracy_score(y_test, model.predict(X_test)), 5)}')
# Modelo 2
model.set_params(**params_2).fit(X_train, y_train)
print(f'Accuracy para Modelo 2 = {round(accuracy_score(y_test, model.predict(X_test)), 5)}')
# Modelo 3
model.set_params(**params_3).fit(X_train, y_train)
print(f'Accuracy para Modelo 3 = {round(accuracy_score(y_test, model.predict(X_test)), 5)}')

Accuracy para Modelo 1 = 0.70733
Accuracy para Modelo 2 = 0.70733
Accuracy para Modelo 3 = 0.70733


In [18]:
params_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [5,6,7],
        'criterion':['entropy','gini']
        }

In [19]:
# Función para buscar los hiperparametros
def apply_search_cv(model, params_grid, X_train, y_train, X_test, y_test, search_method, cv_method, scoring="accuracy", **kwargs):
    grid_cv = search_method(model, params_grid, scoring=scoring, cv=cv_method, **kwargs)
    grid_cv.fit(X_train, y_train)

    print("Mejores parametros:", grid_cv.best_params_)
    print("Mejor score de CV:", grid_cv.best_score_)
    print(f'Accuracy del modelo: {round(accuracy_score(y_test, grid_cv.predict(X_test)), 5)}')


In [20]:
# Grid Search
apply_search_cv(model, params_grid, X_train, y_train, X_test, y_test, search_method=GridSearchCV, cv_method=3)

Mejores parametros: {'colsample_bytree': 0.6, 'criterion': 'entropy', 'gamma': 0.5, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.6}
Mejor score de CV: 0.7214285259345162
Accuracy del modelo: 0.70733


In [21]:
# Randomized Search CV
apply_search_cv(model, params_grid, X_train, y_train, X_test, y_test, search_method=RandomizedSearchCV, cv_method=3)

Mejores parametros: {'subsample': 1.0, 'min_child_weight': 5, 'max_depth': 5, 'gamma': 1.5, 'criterion': 'entropy', 'colsample_bytree': 0.8}
Mejor score de CV: 0.7214285259345162
Accuracy del modelo: 0.70733


In [22]:
# Halving GridSearch CV
apply_search_cv(model, params_grid, X_train, y_train, X_test, y_test, search_method=HalvingGridSearchCV, cv_method=5 ,factor=3)

Mejores parametros: {'colsample_bytree': 0.6, 'criterion': 'gini', 'gamma': 1.5, 'max_depth': 6, 'min_child_weight': 5, 'subsample': 1.0}
Mejor score de CV: 0.7296296296296296
Accuracy del modelo: 0.70733


In [23]:
# Halving Randomized Search
apply_search_cv(model, params_grid, X_train, y_train, X_test, y_test, search_method=HalvingRandomSearchCV, cv_method=3, factor=3)

Mejores parametros: {'subsample': 1.0, 'min_child_weight': 10, 'max_depth': 7, 'gamma': 0.5, 'criterion': 'gini', 'colsample_bytree': 0.6}
Mejor score de CV: 0.7152660346198889
Accuracy del modelo: 0.70733
